In [1]:
#pip install cltk

In [ ]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True )

Mounted at /content/drive


In [ ]:
from bs4 import BeautifulSoup
from nltk.tokenize import word_tokenize, sent_tokenize
import nltk
import cltk
from cltk import NLP
import pandas as pd
import numpy as np
import os
import re

In [ ]:
 nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

Formatting text (XML to Tuple of desired attributes)

In [ ]:
def desoupify(fileloc):
    #Creates soup object
    with open( fileloc, 'r') as f:
        file = f.read()
    xml_soup = BeautifulSoup(file,'xml')

    note_tags = xml_soup.find_all('note')
    for note in note_tags:
        note.decompose()

#grabs text from the object
    text_soup = xml_soup.find_all("p")

    length = len(text_soup)
    if length == 0:
        text_soup = xml_soup.find_all("l")
#joins into text
    text = []
    for x in text_soup:
        text.append(x.text)
    text = ' '.join(text)

    return text

In [ ]:
#Cleans text
def text_clear(text):

#converts all characters to lower case
    string = text.lower()

    string = string.replace("\n"," ")

#removes notes
    string = re.sub('<note.*</note>','',string)

#replaces Kal. with Kalends
    string = string.replace("kal.","kalends")
#cleans out punctuation
    punc = '''()-[]{};:'"\,<>/@,#$%^&,:*_~'''
    for sym in string:
        if sym in punc:
            string = string.replace(sym,"")
    string = string.replace('keyboarding','')
    return string

In [ ]:
#tokenizer
def tokenizer_pipline(string):
    finished = []
    string = sent_tokenize(string)
    for x in string:
        finished.append(word_tokenize(x))

    return finished

In [ ]:
#example of cleaned text
fin = text_clear(desoupify('/content/drive/MyDrive/Final_Project/Ovid/Fasti'))
print(fin[:100])

  tempora cum causis latium digesta per annum lapsaque sub terras ortaque signa canam excipe pacato 


In [ ]:
#establishing NLP pipline and removing all but the LatinStanzaProcess
cltk_nlp = NLP(language="lat")
cltk_nlp.pipeline.processes.pop(-1)
cltk_nlp.pipeline.processes.pop(-2)
cltk_nlp.pipeline.processes.pop(-3)
cltk_nlp.pipeline.processes.pop(1)

‎𐤀 CLTK version '1.1.6'.
Pipeline for language 'Latin' (ISO: 'lat'): `LatinNormalizeProcess`, `LatinStanzaProcess`, `LatinEmbeddingsProcess`, `StopsProcess`, `LatinLexiconProcess`.


cltk.stops.processes.StopsProcess

In [ ]:
print(cltk_nlp.pipeline.processes)

[<class 'cltk.dependency.processes.LatinStanzaProcess'>]


Tuples to Dataframe

In [ ]:
#Creates tuples of desired atrributes
def attributor(directory,period):
    final = []

    for filename in os.listdir(directory):
        f = os.path.join(directory, filename)
        ready = tokenizer_pipline(text_clear(desoupify(f)))
        author = re.findall(r'[A-Za-z]*$',directory)[0]
        for x in ready:
           y = ' '.join(x)
           final.append((author,period,filename,x,y))
    return final

In [ ]:
#Location author directory => df of all texts in dir , Period of work
data = attributor('/content/drive/MyDrive/Final_Project/Arnobius','Patristic')

In [ ]:
# Creating a DataFrame
df = pd.DataFrame(data,columns=['Author','Period','Work','Tokenized Text','Text'])
df.head(1)

In [ ]:
#5000 entries = ~15 minutes
#Lemmatizing
lemmas =[]
count = 0
for x in df['Text']:
  lemmata = cltk_nlp.analyze(text= x).lemmata
  lemmas.append(lemmata)

In [ ]:
#adds lemmas to df
df["Lemmatized Text"] = lemmas

In [ ]:
#only use if work entries appear as "work.xml"
df['Work'] = df['Work'].apply(lambda x: str(x)[:-4])

In [ ]:
author = str(df['Author'][0])

In [ ]:
df.head(2)

,Author,Period,Work,Tokenized Text,Text,Lemmatized Text
0,Arnobius,Patristic,Disputationes adversus gentes,"[patrologia, latina, this, file, was, encoded,...",patrologia latina this file was encoded in tei...,"[patrologia, latina, thus, filius, was, encode..."
1,Arnobius,Patristic,Disputationes adversus gentes,"[roelli, in, 2019, classical, latin, orthography]",roelli in 2019 classical latin orthography,"[roellum, in, 2019, classicalis, latior, ortho..."


In [ ]:
df.to_csv(author +'.csv', sep=',', index=False, encoding='utf-8')